In [38]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")

db = client["conf-con-kathy-durmiendo"] # client.conf

ImportError: cannot import name 'ObjectId' from 'pymongo' (c:\Users\Andy\AppData\Local\Programs\Python\Python311\Lib\site-packages\pymongo\__init__.py)

In [30]:
db.asignaturas.insert_one({
    "nombre": "tortura poli'tica",
    "semanas": 12
})

InsertOneResult(ObjectId('66030fea7051fd4c87fc5cd3'), acknowledged=True)

In [37]:
list(db.asignaturas.find({
    "semanas": 12
}))

[{'_id': ObjectId('66030f207051fd4c87fc5cd1'),
  'asignatura': 'bases de datos',
  'semanas': 12},
 {'_id': ObjectId('66030fe07051fd4c87fc5cd2'),
  'nombre': "tortura poli'tica",
  'semanas': 12},
 {'_id': ObjectId('66030fea7051fd4c87fc5cd3'),
  'nombre': "tortura poli'tica",
  'semanas': 12}]

In [43]:
list(db.asignaturas.find({
    "semanas": 12
}))

[]

In [42]:
db.asignaturas.delete_many({
    "semanas": 12
})

DeleteResult({'n': 2, 'ok': 1.0}, acknowledged=True)

In [8]:
db.accessor_collection.insert_one({"value": "accessor value"})

InsertOneResult(ObjectId('6602dff07051fd4c87fc5cc4'), acknowledged=True)

In [44]:
password_validation = {
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['password'],
        'properties': {
            'password': {
                'bsonType': 'string',
                'description': 'password must be a string and is required'
            }
        }
    }
}
db.create_collection("users", validator=password_validation)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'conf-con-kathy-durmiendo'), 'users')

In [47]:
db.users.insert_one({
    "name": "Andy",
    "age": 24,
    "password": "andy123"
})

InsertOneResult(ObjectId('6603134b7051fd4c87fc5cd6'), acknowledged=True)

In [48]:
total_with_vat_validation = { # totalWithVAT = total * (1 + VAT)
    '$expr': {
        '$eq': [
            "$totalWithVAT",
            {
                '$multiply': [
                    "$total",
                    {'$sum': [1, "$VAT"]}
                ]
            }
        ]
    }
}
db.create_collection("orders", validator=total_with_vat_validation)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'conf-con-kathy-durmiendo'), 'orders')

In [49]:
# incorrecto
db.orders.insert_one({
    "total": 141,
    "VAT": 0.20,
    "totalWithVAT": 169
})

WriteError: Document failed validation, full error: {'index': 0, 'code': 121, 'errmsg': 'Document failed validation', 'errInfo': {'failingDocumentId': ObjectId('660314597051fd4c87fc5cd7'), 'details': {'operatorName': '$expr', 'specifiedAs': {'$expr': {'$eq': ['$totalWithVAT', {'$multiply': ['$total', {'$sum': [1, '$VAT']}]}]}}, 'reason': 'expression did not match', 'expressionResult': False}}}

In [50]:
# correcto
db.orders.insert_one({
    "total": 141,
    "VAT": 0.20,
    "totalWithVAT": 169.2
})

InsertOneResult(ObjectId('660314757051fd4c87fc5cd8'), acknowledged=True)

In [51]:
discounted_price_validator = {
    "$and": [
        {
            "$expr": {
                "$lt": ["$lineItems.discountedPrice", "$lineItems.price"]
            }
        },
        {
            "$jsonSchema": {
                "bsonType": "object",
                "properties": {
                    "items": {"bsonType": "array"}
                }
            }
        }
    ]
}

# Create the 'sales' collection with the specified validator
db.create_collection("sales", validator=discounted_price_validator)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'conf-con-kathy-durmiendo'), 'sales')

In [22]:
# incorrecto
db.sales.insert_one({
    "lineItems": {
        "discountedPrice": 23,
        "price": 19
    }
})

WriteError: Document failed validation, full error: {'index': 0, 'code': 121, 'errmsg': 'Document failed validation', 'errInfo': {'failingDocumentId': ObjectId('6602f58b7051fd4c87fc5ccf'), 'details': {'operatorName': '$and', 'clausesNotSatisfied': [{'index': 1, 'details': {'operatorName': '$jsonSchema', 'schemaRulesNotSatisfied': [{'operatorName': 'properties', 'propertiesNotSatisfied': [{'propertyName': 'items', 'details': [{'operatorName': 'bsonType', 'specifiedAs': {'bsonType': 'array'}, 'reason': 'type did not match', 'consideredValue': 23, 'consideredType': 'int'}]}]}]}}]}}}

In [ ]:
# correcto
db.sales.insert_one({
    "lineItems": {
        "discountedPrice": 23,
        "price": 24
    },
    # "items": 23
})

In [ ]:
# correcto
db.sales.insert_one({
    "lineItems": {
        "discountedPrice": 23,
        "price": 24
    },
    "items": ["car", "toy"]
})